# Feature Classificaiton
Create a system to better analyze each feature by its meta data. This will be a method to seperate 3rd party data by its source or by sales features by its type of analysis.
This script will look through all columns from 6 divisions to attempt to get complete list that can be used for all models for all the monthly Models by pulling sample from each division

## Version:
0.1 - Created initial script based on previous Feature classification Script.    
0.2 - Updated S3 bucket location to use the new Obmarketing Zone in Prod. Changed to get features to get as close to column order as possible   
0.3 - Used first run for entire 25,000 columns which should provide all classifications. Includes row for type of mean replacement for easy preprocessing

In [ ]:
import sys
# !conda install --yes --prefix {sys.prefix}  "pandas >=1.1.0" "s3fs>=0.4.2" regex boto3

In [ ]:
#Import all standard packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pickle
import seaborn as sns
import boto3

#Import new packages
import os
import regex as re

In [ ]:
#use this kernal option to enable me to see the value of maultiple statements at once
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

## Load in the Feature list
Grab in a sample from each division

In [ ]:
file_location = "s3://bluestembrands-bi-published-zone-prod/orchard/marketing/house/retro/catalog_refresh/end_date=20200703/"

In [ ]:
Sample = pd.read_parquet(file_location,
                    filters=[('household_key' , '=', '01953405455')],
                    engine='pyarrow')

In [ ]:
Sample

In [ ]:
#Get a dataframe of all columns
all_columns = pd.DataFrame(  data =  np.unique(np.array( ['individual_key'] + list(Sample.columns) + ['shipped_qty'] )), columns=["col_name"] )
all_columns

In [ ]:
len(all_columns)

## Load in Id Features and Remove
These features will identify who the person is and some details about them. Remove these columsn

In [ ]:
id_features = pd.read_csv("s3://bluestembrands-bi-obmarketing-zone-prod/Feature_Classification/id_column_descriptions.csv")

id_features

In [ ]:
#View what ID features are within this dataset
all_columns.merge(id_features, how= 'inner', on = ['col_name'] )

In [ ]:
no_id_columns = all_columns.set_index('col_name').drop( labels = set(id_features['col_name']).intersection(all_columns['col_name']) , axis = 0).reset_index()

## Load in 3rd party Metadata

In [ ]:
raw_third_party = pd.read_csv("s3://bluestembrands-bi-obmarketing-zone-prod/Feature_Classification/3rd_Party_Data.csv")

#Define type as 3rd party
raw_third_party['Type'] = "3rd Party"
#Define Aggregation Type as "3rd Party Entering"
raw_third_party['Aggregation Type'] = "3rd Party"

In [ ]:
third_party_meta = raw_third_party.drop(labels = ["data_type", "upper(col_name)"], axis = 1)

# third_party_meta

In [ ]:
#Remove the _rc from the column names which is an add on
third_party_meta_add = third_party_meta.copy()

third_party_meta_add["col_name"] =  third_party_meta_add["col_name"].apply( lambda x: x+ '_rc' ) 

# third_party_meta_add

In [ ]:
third_party_meta = pd.concat([third_party_meta, third_party_meta_add])

third_party_meta.head()

## Determine type of internal data
Will be all features not in 3rd party

### Load in features

In [ ]:
df = no_id_columns.merge(third_party_meta, how = "left", on = "col_name" )

# df.head()

In [ ]:
#Define internal as anything that is missing. Define as Type Internal
internal_features = df[df["Type"].isna() ].copy()

internal_features['Type'] = "Internal"

internal_features

In [ ]:
len(internal_features)

### External Data Analysis
Define all external data as all of those who were part of 3rd party data above

In [ ]:
#Define as a copy
external_features = df[df["Type"].notna() ].copy()

external_features

In [ ]:
external_features['description'] = external_features['description'].apply( lambda x: x.title() )

#Define the replacement type of mean for all 3rd party data
external_features['replacement_type'] = 'mean'

external_features

In [ ]:
#View the source
external_features.groupby("Source").count().sort_values('col_name', ascending= False)

### Manual Data Internal
Data without a good systematic name

In [ ]:
#Load in the manually described internal features from customer aggs
customer_aggs = pd.read_csv("s3://bluestembrands-bi-obmarketing-zone-prod/Feature_Classification/manual_internal_features.csv")

# customer_aggs

In [ ]:
#Get Everything title case
customer_aggs['description'] = customer_aggs['description'].apply( lambda x: x.title() )

customer_aggs

In [ ]:
#Drop the manual
automatic_generated = internal_features.set_index('col_name').drop( labels = set(internal_features['col_name']).intersection(customer_aggs['col_name']) , axis = 0).reset_index()

len(automatic_generated)
automatic_generated

## Import Dictionary
Create manual dictionaries and load others

In [ ]:
division_dict = { "apl": "Appleseeds", 
                 "bla women": "Blair Women", 
                 "bla men": "Blair Men",
                 "hab women": "Haband Women", 
                 "hab men": "Haband Men",
                "drd": "Drapers & Damons"}

In [ ]:
recency_dict = {"0 3": "0-3"
                ,"3 6": "3-6"
                ,"0 6": "0-6" 
                ,"6 12": "6-12" 
                ,"12 24": "12-24" 
                ,"24 36": "24-36"
               ,"36 48": "36-48"
               ,"0 48": "0-48"}

#### Category Dictionary

In [ ]:
#Load in category as dataframe
category_df = pd.read_csv("s3://bluestembrands-bi-obmarketing-zone-prod/Feature_Classification/Category_Code_Lookup.csv")

#Drop the Null or NaN in product_code
category_df.dropna(subset = ['product_code'], inplace = True )

#Lowercase all product codes
category_df['product_code'] = category_df['product_code'].apply( lambda x: x.lower() )

#Define Category Dictionary
category_dict = dict(zip(category_df.product_code, category_df.product_category))

# category_dict

#### Subcategory Dictionary

In [ ]:
#Load in category as dataframe
subcategory_df = pd.read_csv("s3://bluestembrands-bi-obmarketing-zone-prod/Feature_Classification/Subcategory_rollup.csv")

#Drop the Null or NaN in product_code
subcategory_df.dropna(subset = ['product_subcategory_number'], inplace = True )

# subcategory_df

In [ ]:
#Determine the full description by adding ctegory to subcategory 
subcategory_df['full_description'] = subcategory_df['product_subcategory_description'] + ' (' + subcategory_df['product_category'] +')'

#Add in sub in front of all numbers for an easier lookup
subcategory_df['product_subcategory_number'] = 'sub ' + subcategory_df['product_subcategory_number'].apply( lambda x: str(x)  )

#Define Dictionary
subcategory_dict = dict(zip(subcategory_df.product_subcategory_number, subcategory_df.full_description))

# subcategory_dict

### Day Columns
Determine description for Divisions. This will be all columsn that contain the word day

In [ ]:
#Define Day From columns that start with Days
day_from_features = automatic_generated[automatic_generated['col_name'].str.contains("days_*", regex = True)].copy()

# day_from_features

In [ ]:
#Define the Source as customer transactions
day_from_features['Source'] = "Customer Transactions"
#Define aggregation type as "Date Difference"
day_from_features['Aggregation Type'] = "Date Difference"

#Define The replacement type as max
day_from_features['replacement_type'] = 'max'

# day_from_features

#### Description Determination

In [ ]:
#Define the description as col_name without _ or make everything be from
day_from_features['description'] = day_from_features['col_name'].apply( lambda x: re.sub( "since", "from", re.sub( "_", " ",  x ))  )

# day_from_features

In [ ]:
#Replace abbreviations with dictionary values and title case everything
day_from_features['description'] = day_from_features['description'].replace(division_dict, regex=True).apply(lambda x: x.title())

day_from_features

In [ ]:
removed_days = automatic_generated.set_index('col_name').drop( labels = day_from_features['col_name'].tolist(), axis = 0).reset_index()

len(removed_days)
removed_days.head()

### Define Subcategory Features

In [ ]:
#Define subcategory by everything that starts with Sub
subcategory_features = removed_days[removed_days['col_name'].str.contains("sub_*", regex = True)].copy()

# subcategory_features

In [ ]:
#Define the Source
subcategory_features['Source'] = "Customer Transactions"

#Define the Aggregation Type
subcategory_features['Aggregation Type'] = "Subcategory Level Sum"

#Define The replacement type as zero
subcategory_features['replacement_type'] = 'zero'

#### Description Determination

In [ ]:
#Replace replace "_" with " "
subcategory_features['description'] = subcategory_features['col_name'].apply( lambda x: re.sub( "_", " ",  x ))  

# subcategory_features

In [ ]:
#Put a dash between Months and replace ending M with Months using dictionary
subcategory_features['description'] = subcategory_features['description'].apply(lambda x: re.sub( "m$", " Months",x )).replace(recency_dict, regex=True) 

# subcategory_features

In [ ]:
#replace all divisions with division names
subcategory_features['description'] = subcategory_features['description'].replace(division_dict, regex=True)

# subcategory_features

In [ ]:
#replace all sub values with full Names
subcategory_features['description'] = subcategory_features['description'].replace(subcategory_dict, regex=True).apply( lambda x: x.title() )

subcategory_features

In [ ]:
#Remove subcateogyr values from the removed days set
remove_subcategory = removed_days.set_index('col_name').drop( labels = subcategory_features['col_name'].tolist(), axis = 0).reset_index()

len(remove_subcategory)
remove_subcategory

### Division Variables 

In [ ]:
division_features = remove_subcategory[remove_subcategory['col_name'].str.contains("^(apl)|^(bla)|^(hab)|^(drd)|^(division)_*", regex = True)].copy()

# division_features

In [ ]:
#Define the Source
division_features['Source'] = "Customer Transactions"

#Define the Aggregation Type
division_features['Aggregation Type'] = "Division Level Sum"

#Define The replacement type as zero
division_features['replacement_type'] = 'zero'

#### Description Determination

In [ ]:
#Replace replace "_" with " "
division_features['description'] = division_features['col_name'].apply( lambda x: re.sub( "_", " ",  x ))  

# division_features

In [ ]:
#Put a dash between Months and replace ending M with Months using dictionary
division_features['description'] = division_features['description'].apply(lambda x: re.sub( "m$", " Months",x )).replace(recency_dict, regex=True) 

# division_features

In [ ]:
#replace all divisions with division names and title case
division_features['description'] = division_features['description'].replace(division_dict, regex=True).apply( lambda x: x.title() )

division_features

In [ ]:
remove_division = remove_subcategory.set_index('col_name').drop( labels = division_features['col_name'].tolist(), axis = 0).reset_index()

remove_division

### Category Variables 

In [ ]:
#Define category by anything that starts with three values that isn't apl, bla, hab, drd which can be first value
category_features = remove_division[remove_division['col_name'].str.contains("^[\\w]{3}_", regex = True)].copy()

category_features

In [ ]:
#Define the Source
category_features['Source'] = "Customer Transactions"

#Define the Aggregation Type
category_features['Aggregation Type'] = "Category Level Sum"

#Define The replacement type as zero
category_features['replacement_type'] = 'zero'

#### Description Determination

In [ ]:
#Replace replace "_" with " "
category_features['description'] = category_features['col_name'].apply( lambda x: re.sub( "_", " ",  x ))  

# category_features

In [ ]:
#Put a dash between Months and replace ending M with Months using dictionary
category_features['description'] = category_features['description'].apply(lambda x: re.sub( "m$", " Months",x )).replace(recency_dict, regex=True) 

# category_features

In [ ]:
#replace all divisions with division names
category_features['description'] = category_features['description'].replace(division_dict, regex=True)

# category_features

In [ ]:
#Replace Category with Dictionary values from category
category_features['description'] = category_features['description'].replace(category_dict, regex=True).apply( lambda x: x.title() )

category_features

In [ ]:
remove_category = remove_division.set_index('col_name').drop( labels = category_features['col_name'].tolist(), axis = 0).reset_index()

len(remove_category)
remove_category

### Source Code
Source code count is a division variable but done differently

In [ ]:
#Define category by anything that starts with three values that isn't apl, bla, hab, drd which can be first value
source_code_features = remove_category[remove_category['col_name'].str.contains("^(source_code)_*", regex = True)].copy()

# source_code_features

In [ ]:
#Define the Source
source_code_features['Source'] = "Promos Table"

#Define the Aggregation Type
source_code_features['Aggregation Type'] = "Promo Level Sum"

#Define The replacement type as zero
source_code_features['replacement_type'] = 'zero'

#### Description Determination

In [ ]:
#Replace replace "_" with " "
source_code_features['description'] = source_code_features['col_name'].apply( lambda x: re.sub( "_", " ",  x ))  

# source_code_features

In [ ]:
#Put a dash between Months and replace ending M with Months using dictionary
source_code_features['description'] = source_code_features['description'].apply(lambda x: re.sub( "m$", " Months",x )).replace(recency_dict, regex=True) 

# source_code_features

In [ ]:
#replace all divisions with division names
source_code_features['description'] = source_code_features['description'].replace(division_dict, regex=True).apply( lambda x: x.title() )

source_code_features

In [ ]:
remove_source_code = remove_category.set_index('col_name').drop( labels = source_code_features['col_name'].tolist(), axis = 0).reset_index()

len(remove_source_code)
remove_source_code

### Customer Level Summary

In [ ]:
customer_summary_features = remove_source_code.copy()

In [ ]:
#Define the Source
customer_summary_features['Source'] = "Customer Transactions"

#Define the Aggregation Type
customer_summary_features['Aggregation Type'] = "Category Level Sum"

#Add in replacement type
customer_summary_features['replacement_type'] = 'zero'

#### Description Determination

In [ ]:
#Replace replace "_" with " "
customer_summary_features['description'] = customer_summary_features['col_name'].apply( lambda x: re.sub( "_", " ",  x ).title() )  

customer_summary_features

## Join all Feature Types 
Will get an entire metadata analsis set

In [ ]:
all_features_meta = pd.concat([external_features, customer_aggs, day_from_features, subcategory_features, division_features, category_features, source_code_features, customer_summary_features   ] )

all_features_meta.head()

In [ ]:
### Create dataframe of sort order for aggregation types 
aggregation_sort_order = {
    'Customer Level Sum': 1,
    'Date Difference': 2,
    'Promo Level Sum':3,
    'Division Level Sum':4,
    'Category Level Sum': 5,
    'Subcategory Level Sum':6,
    '3rd Party':7 }

aggregation_sort_order

In [ ]:
#Add in these sort orders
all_features_meta['group_sort_order'] = all_features_meta['Aggregation Type'].apply( lambda x:aggregation_sort_order[x]  )

#Sort by the sort sort, source and column names
all_features_meta.sort_values( ['group_sort_order', 'Source', 'col_name' ], inplace = True  )

## Create a column called sort order whihc is just enumeriation of this sorted order via a new index
all_features_meta.reset_index( drop = True, inplace = True )
all_features_meta['sort_order'] = all_features_meta.index + 1

all_features_meta

In [ ]:
all_features_meta[[ 'Aggregation Type' ]].drop_duplicates().reset_index(drop = True)

In [ ]:
#Check and see if there are any missing columns
all_features_meta.isna().any()

In [ ]:
#Left Join to ensure that we have
test = pd.DataFrame(no_id_columns, columns= ["col_name"] ).merge(all_features_meta, how = "left", on = "col_name" )

#Ensure that everything is working correctly
test.head()

In [ ]:
#Ensure there are no missing
test[test['description'].isna()]

In [ ]:
#Write out the features. Note that the index will be the approximate order based on ingestion for the test set since will be original order
# all_features_meta.to_csv("s3://bluestembrands-bi-obmarketing-zone-prod/Feature_Classification/all_features_metadata.csv", index = False)